
## Nnwdaf_Analytics_Subscription_Subscribe

In [ ]:
#EC2 Nnwdaf_Analytics_Subscription_Subscribe
from flask import Flask, request, jsonify
from flask_cors import CORS
from sklearn.preprocessing import MinMaxScaler
import boto3, json
import joblib
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable
from io import BytesIO
import numpy as np
from urllib.parse import urlparse
from GRUModel import GRUModel
app=Flask(__name__)
CORS(app)
@app.route('/',methods=['POST'])
def handler():
    client = boto3.client("s3")
    bytes_ = BytesIO()
    parsed_s3 = urlparse("s3://thesisntpu20220513/1009680truevalue_0714.npy")
    client.download_fileobj(
        Fileobj=bytes_, Bucket=parsed_s3.netloc, Key=parsed_s3.path[1:]
    )
    bytes_.seek(0)
    data=np.load(bytes_, allow_pickle=True)
    gruencoder=joblib.load("/home/ec2-user/environment/python-nwdaf-3.14/gruencoder.pkl")
    grulearner=joblib.load("/home/ec2-user/environment/python-nwdaf-3.14/grulearner.pkl")
    sc = MinMaxScaler()
    testX = sc.fit_transform(data)
    testX=testX.reshape(-1,1,1)
    testX = Variable(torch.Tensor(np.array(testX)))
    test = grulearner(gruencoder(testX))
    test = test.data.numpy()
    test = sc.inverse_transform(test)
    bytes_2= BytesIO()
    np.save(bytes_2, test, allow_pickle=True)
    bytes_2.seek(0)
    parsed_s3_u = urlparse("s3://thesisntpu20220513/1009680truevalue_0714.npy")
    client.upload_fileobj(
         Fileobj=bytes_2, Bucket=parsed_s3_u.netloc, Key=parsed_s3_u.path[1:]
     )
    # output=json.loads(json.dumps(test, default=str))
    response = {'statusCode': 200, 'body' : "OK"}
    return(response)
if __name__ =='__main__':
    app.run(host='0.0.0.0',port=3001,debug=True)

